In [0]:
import librosa
import librosa.display as libdisplay
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# **import Library Penting **

In [0]:
max_pad_len = 388
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset_path = '/content/drive/My Drive/SOUND/set_a/'
metadata = pd.read_csv('/content/drive/My Drive/SOUND/DATASET_A_NEW.csv')

features = []

In [0]:
len(mfccs)

40

In [0]:
max_pad_len - mfccs.shape[1]

-214

In [13]:
import glob, os
os.chdir("/content/drive/My Drive/SOUND/set_a/")


MAX_LEN_MFCC = []

for file in glob.glob("*.wav"):
    #print(file)
    #extract_features(file)

    audio,sample_rate = librosa.load(file, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #print(mfccs.shape[1])
    MAX_LEN_MFCC.append(mfccs.shape[1])

print(max(MAX_LEN_MFCC)   )

# dari data dapet 388 len nya 

388


In [0]:
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(dataset_path),str(row['fname']))
    
    class_label = row['fname']
    data = extract_features(file_name)
    
    features.append([data,class_label])

In [32]:
DATAX = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(DATAX), ' files')

Finished feature extraction from  496  files


In [0]:
str(DATAX['class_label'][0]).split('__')[0]

LABEL = []

for i in range(len(DATAX)):
  str(DATAX['class_label'][i]).split('__')[0]
  LABEL.append(str(DATAX['class_label'][i]).split('__')[0])


In [0]:
DATAX['LABEL'] = LABEL

NEW_DATAX = DATAX.drop(columns=['class_label'])

NEW_DATAX.to_csv(r'NEW_DATAX.csv')

In [50]:
NEW_DATAX

,feature,LABEL
0,"[[-629.3815027521428, -614.3528398171517, -616...",artifact
1,"[[-990.9772750518831, -990.1641333446495, -989...",artifact
2,"[[-632.0832226192613, -622.4774297438613, -623...",artifact
3,"[[-322.1109746901032, -295.3795520831543, -277...",artifact
4,"[[-294.6576987096496, -292.31416581691946, -28...",artifact
...,...,...
491,"[[-522.8667459421804, -491.1161367555086, -468...",normal
492,"[[-497.0974694148711, -494.25662561002684, -49...",normal
493,"[[-510.270796132117, -517.8475036795197, -516....",normal
494,"[[-502.076714187306, -493.8695590658039, -495....",normal


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, utils
import matplotlib.pyplot as plt
from sklearn import metrics 

In [0]:
from sklearn.preprocessing import LabelEncoder

X = np.array(NEW_DATAX.feature.tolist())
y = np.array(NEW_DATAX.LABEL.tolist())

In [0]:
le = LabelEncoder()
yy = tf.keras.utils.to_categorical(le.fit_transform(y))

In [0]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [125]:
#Untuk menentukan column nanti di model

print('NEW_DATAX shape >> {}'.format(NEW_DATAX.shape))
print('len X_train >> {}'.format(len(x_train)))
print('len X_test >> {}'.format(len(x_test)))

# untuk dapetin num_rows, num_clumns d
print('X_train shape >> {}'.format(x_train.shape))

# len(mfccs) --> nanti kita gunakan untuk num_rows
# max(MAX_LEN_MFCC) --> dibunakan untuk num_columns & max_pad_len pada fungsi extract_features
print('Num_Rows >> {}'.format(len(mfccs)))
print('Num_Columns >> {}'.format(max(MAX_LEN_MFCC)))

NEW_DATAX shape >> (496, 2)
len X_train >> 396
len X_test >> 100
X_train shape >> (396, 40, 388, 1)
Num_Rows >> 40
Num_Columns >> 388


# **Mempersiapkan nilai untuk Train & Test**

In [0]:
num_rows = 40
num_columns = 388
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

#  Membuat MODEL (**CNN MODEL**) 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dense(num_labels, activation='softmax'))

# **Meng-compile Model**

In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# **Summary Model**

In [87]:
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 39, 387, 16)       80        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 19, 193, 16)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 19, 193, 16)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 192, 32)       2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 9, 96, 32)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 9, 96, 32)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 95, 64)        

# **Training Model** ( dengan beberapa kali iterasi/ epoch )


model disimpan dengan nama 'sound_classification_Bagus.hdf5' *bisa disesuaikan sik


In [114]:
#tf.keras.callbacks.ModelCheckpoint

from datetime import datetime 

#num_epochs = 500
#num_batch_size = 128

#num_epochs = 500
#num_batch_size = 256

#num_epochs = 100
#num_batch_size = 256

num_epochs = 15
num_batch_size = 256

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='sound_classification_Bagus.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 396 samples, validate on 100 samples
Epoch 1/15
256/396 [==================>...........] - ETA: 1s - loss: 0.0380 - acc: 0.9805
Epoch 00001: val_loss improved from inf to 0.24852, saving model to sound_classification_Bagus.hdf5
396/396 [==============================] - 5s 12ms/sample - loss: 0.0434 - acc: 0.9773 - val_loss: 0.2485 - val_acc: 0.9000
Epoch 2/15
256/396 [==================>...........] - ETA: 1s - loss: 0.0417 - acc: 0.9844
Epoch 00002: val_loss did not improve from 0.24852
396/396 [==============================] - 4s 10ms/sample - loss: 0.0423 - acc: 0.9848 - val_loss: 0.2813 - val_acc: 0.8600
Epoch 3/15
256/396 [==================>...........] - ETA: 1s - loss: 0.0445 - acc: 0.9766
Epoch 00003: val_loss did not improve from 0.24852
396/396 [==============================] - 4s 10ms/sample - loss: 0.0389 - acc: 0.9823 - val_loss: 0.2672 - val_acc: 0.8800
Epoch 4/15
256/396 [==================>...........] - ETA: 1s - loss: 0.0322 - acc: 0.9883
Epoch 00004: val

# **Melihat Akurasi**
dalam training pada case ini dilakukan beberapa kali hingga mendapatkan akurasi mendekati 0.9


In [115]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.95454544
Testing Accuracy:  0.9


# **Membuat Fungsi Untuk Prediksi**

In [0]:
def buat_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [116]:
#/content/drive/My Drive/SOUND/set_b/Bunlabelledtest__101_1305030823364_A.wav

filename = '/content/drive/My Drive/SOUND/set_a/Aunlabelledtest__201012172010.wav' 
buat_prediction(filename)

The predicted class is: artifact 

artifact 		 :  0.99999845027923583984375000000000
extrahls 		 :  0.00000159803221322363242506980896
murmur 		 :  0.00000000577820324920708117133472
normal 		 :  0.00000000000096079947383559405338


In [117]:
filename = '/content/drive/My Drive/SOUND/set_a/Aunlabelledtest__201108222257.wav' 
buat_prediction(filename)

The predicted class is: murmur 

artifact 		 :  0.00000000000049246267870908799758
extrahls 		 :  0.00000000000000670764247436201645
murmur 		 :  0.99849021434783935546875000000000
normal 		 :  0.00150972185656428337097167968750
